In [6]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from ai_cdss.data_loader import DataLoader
from ai_cdss.data_processor import DataProcessor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
rgs_mode = "app"
scoring_weights = [1,1,1]
ewma_alpha = 0.5

In [8]:
patient_list = [
    775,  787,  788, 1123, 1169, 1170, 1171, 1172, 1173, 1983, 2110, 2195,
    2955, 2956, 2957, 2958, 2959, 2960, 2961, 2962, 2963, 3081, 3229, 3318, 3432
]

In [9]:
loader = DataLoader(rgs_mode=rgs_mode)
processor = DataProcessor(weights=scoring_weights, alpha=ewma_alpha)

In [10]:
session = loader.load_session_data(patient_list=patient_list)
timeseries = loader.load_timeseries_data(patient_list=patient_list)
ppf = loader.load_ppf_data(patient_list=patient_list)

Database engine created successfully


INFO:ai_cdss.data_loader:Session data loaded successfully.


Database engine closed
Database engine created successfully
Database engine closed
Database engine created successfully


INFO:ai_cdss.data_loader:PPF data loaded successfully.


Database engine closed


In [11]:
scoring = processor.process_data(session_data=session, timeseries_data=timeseries, ppf_data=ppf)

#### CDSS

In [15]:
from ai_cdss.cdss import CDSS

In [16]:
protocol_similarity = loader.load_protocol_similarity()

INFO:ai_cdss.data_loader:Protocol similarity data loaded successfully.


In [17]:
cdss = CDSS(
    scoring=scoring
)

In [ ]:
cdss.decide_prescription_swap(775)

In [ ]:
excl = cdss.get_prescriptions(775).PROTOCOL_ID.to_list()

In [ ]:
excl

In [ ]:
patient_id = 775
protocol_id = 227

In [ ]:
score_row = scoring[
    (scoring["PATIENT_ID"] == patient_id) & (scoring["PROTOCOL_ID"] == protocol_id)
]

In [18]:
for patient_id in patient_list:
    recommendation = cdss.recommend(patient_id=patient_id, protocol_similarity=protocol_similarity)
    break

In [ ]:
cdss.get_prescriptions(patient_id)

In [19]:
recommendation.to_parquet("recommendation.parquet")

['BARTHEL',
 'ASH_PROXIMAL',
 'MA_DISTAL',
 'FATIGUE',
 'VAS',
 'FM_A',
 'FM_B',
 'FM_C',
 'FM_D',
 'FM_TOTAL',
 'ACT_AU',
 'ACT_QOM']